In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
import os
from sklearn.preprocessing import LabelEncoder


new_datasets_path:str = ""
image_path:str = ""
root: str = ""
dataset_path : str = ""
Save_path = ""
Model_Path = ""

In [ ]:
os.chdir(new_datasets_path)
DataSet = pd.DataFrame()
DataSet = pd.read_csv("Final_Dataset.csv")

In [28]:
b_ids = DataSet['breath_id']
DataSet.drop(DataSet.columns[[0,22]],axis=1,  inplace=True) 
DataSet.head()


,u_in_std,pressure_std,Polynomial_Area,Hull_Area,ft_TV_1,ft_TV_2,ft_TV_3,ft_TV_4,ft_TV_5,ft_P_1,...,ft_P_3,ft_P_4,ft_P_5,Perimeter,Width,Height,Centroid_X,Centroid_Y,Curvature,Cluster
0,0.000000,0.000000,282.157479,350.167422,484.607933,183.391093,68.468914,56.159057,51.404353,1082.775828,...,164.998616,67.671789,116.198749,122.759376,18.353980,22.777895,6.057599,13.534698,11.190253,0
1,0.000000,0.000000,28.139413,74.872299,300.129994,60.284442,68.542576,55.210521,40.204153,642.684398,...,52.755145,37.449278,53.714602,65.228595,8.975885,9.982905,3.751625,8.033555,13.949885,1
2,-1.221946,-0.984998,218.777887,272.316931,477.517065,167.435953,75.304841,40.280884,57.576735,1046.921734,...,175.425250,41.169516,109.287272,111.338845,16.383765,21.793665,5.968963,13.086522,4.491883,0
3,0.000000,0.000000,31.145178,15.969105,274.582413,79.187200,56.413160,32.779015,28.114756,548.198314,...,28.938086,14.786057,1.780922,38.983140,5.990302,3.936920,3.432280,6.852479,254.399679,1
4,0.000000,0.000000,237.400212,301.631449,434.662665,101.799568,154.094102,80.326598,43.306234,972.401460,...,279.277437,135.713795,30.636842,105.555013,13.125972,27.417837,5.433283,12.155018,11.018074,0


In [29]:
X_Features = DataSet.drop(columns='Cluster')
y_labels = DataSet["Cluster"]

X_trainSet, X_testSet, Y_trainSet, Y_testSet = train_test_split(X_Features, y_labels, test_size=0.2, random_state=42)

In [30]:
xgb_train = xgb.DMatrix(X_trainSet, label = Y_trainSet, enable_categorical = True)
xgb_test = xgb.DMatrix(X_testSet, label = Y_testSet, enable_categorical = True)

In [39]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
DataSet['EncodedCluster'] = encoder.fit_transform(DataSet['Cluster'])
xgb_train = xgb.DMatrix(X_trainSet, label=DataSet.loc[X_trainSet.index, 'EncodedCluster'])
xgb_test = xgb.DMatrix(X_testSet, label=DataSet.loc[X_testSet.index, 'EncodedCluster'])

params = {
    'objective': 'multi:softmax',
    'num_class': DataSet['EncodedCluster'].nunique(),
    'max_depth': 50,
    'learning_rate': 0.8
}

model = xgb.train(params=params,dtrain=xgb_train,num_boost_round=20000)

In [40]:
y_test = DataSet.loc[X_testSet.index, 'EncodedCluster']

In [41]:
Nostradamus = model.predict(xgb_test)
accuracy = accuracy_score(y_test, Nostradamus)
ClassRep = classification_report(y_test,Nostradamus)
Conf_Matrix = confusion_matrix(y_test, Nostradamus)

/home/molderon/.local/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/molderon/.local/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/molderon/.local/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [42]:
print('Accuracy of the model is:', accuracy*100)

Accuracy of the model is: 99.78031634446397


In [43]:
os.chdir(Model_Path)
model.save_model('PAV_XGB.bin')

In [44]:
ClassRep

'              precision    recall  f1-score   support\n\n           0       0.00      0.00      0.00         1\n           1       1.00      1.00      1.00      1381\n           2       1.00      1.00      1.00       781\n           3       0.50      1.00      0.67         1\n           4       1.00      0.50      0.67         2\n           7       0.89      0.89      0.89         9\n           8       1.00      1.00      1.00         2\n           9       1.00      0.60      0.75         5\n          10       0.97      1.00      0.98        94\n\n    accuracy                           1.00      2276\n   macro avg       0.82      0.78      0.77      2276\nweighted avg       1.00      1.00      1.00      2276\n'

In [28]:
Conf_Matrix

array([[  11,    0,    0,    0,    0],
       [   0, 1381,    0,    0,    0],
       [   0,    0,  781,    0,    0],
       [   0,    0,    0,    1,    1],
       [   0,    1,    0,    0,  100]])